# IMPORT

In [1]:
import pandas as pd
import json
import os
from selectolax.parser import HTMLParser
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import glob

from datetime import date

In [2]:
today = date.today()

In [3]:
input_24 = pd.read_excel(f'Output/2024-06-07_esiti_positivi.xlsx')

In [4]:
input_24= input_24[input_24['esito']=='f24']
len(input_24)

# FUNCTIONS

In [6]:
def apply_mapping(value):
    if 'DECRETO INGIUNTIVO' in value:
        return 'DI - DECRETO INGIUNTIVO'
    elif 'ESECUZIONE MOBILIARE' in value:
        return 'EM - ESECUZIONE MOBILIARE'
    elif 'ORDINANZA' in value:
        return 'OR - ORDINANZA'
    elif 'ATTO VARIO' in value:
        return 'AV - ATTO VARIO'
    else:
        return None  # Default value if no match is found

def apply_mapping_tipo(value):
    if 'DI - DECRETO INGIUNTIVO' in value:
        return 'DECRETO'
    elif 'EM - ESECUZIONE MOBILIARE' in value:
        return 'PPT'
    elif 'OR - ORDINANZA' in value:
        return 'PPT'
    elif 'ATTO VARIO' in value:
        return 'DECRETO'
    else:
        return None    

# Apply the mapping to create the 'natura' column
input_24['natura'] = input_24['atto'].apply(apply_mapping)
input_24['tipo'] = input_24['natura'].apply(apply_mapping_tipo)

In [7]:
def extract_numero(value):
    numero_match = re.search(r'n\. (\d+)', value)
    if numero_match:
        return numero_match.group(1)
    else:
        return None

def extract_anno(value):
    anno_match = re.search(r'del (\d+)', value)
    if anno_match:
        return anno_match.group(1)
    else:
        return None

# Apply the functions to create the 'numero' and 'anno' columns
input_24['numero'] = input_24['atto'].apply(extract_numero)
input_24['anno'] = input_24['atto'].apply(extract_anno)

# PULIZIA INPUT

In [8]:
# aggiungiamo la p.iva/cf del cliente e denominazione (?)
cf_piva_clienti = pd.read_excel('Resources/cf_piva_f24.xlsx', dtype={'CF': str})
cf_piva_clienti['Studio Affidatario'] = cf_piva_clienti['Studio Affidatario'].str.lower()
cf_piva_clienti['Studio Affidatario']  = (cf_piva_clienti['Studio Affidatario'] 
                .str.replace('.', '', regex=False)
                .str.replace('-', '', regex=False)
                .str.strip()
                )
input_24['Cliente'] = input_24['Cliente'].str.lower()
input_24 = pd.merge(input_24, cf_piva_clienti, left_on='Cliente',right_on='Studio Affidatario', how='left')
input_24.drop(columns=['Studio Affidatario'], inplace=True)
input_24['Cliente'] = input_24['Cliente'].str.lower()

In [9]:
input_24 = input_24.drop_duplicates()

In [10]:
# creare colonne nome e cognome da debitore con queste regole:
# cognome è il cliente (denominazione completa)    ? 
# nome è il cognome del debitore 
#   se il cognome è 'de la cruz' usiamo il  copdice sotto per prendere almeno 'de la' 

def split_name(debitore):
    words = debitore.split()
    if len(words[0]) <=5:
        nome = ' '.join(words[0:2])
        if len(nome) <=5 : 
            nome = ' '.join(words[0:3])
    else:
        nome = words[0]
    return nome

# Apply the function to the 'debitore' column
input_24['nome'] = input_24['debitore'].apply(split_name)

In [11]:
def clean_string(s):
    replacements = {
        "'": " ",
        "è": "e",
        "é": "e",
        "à": "a",
        "ì": "i",
        "ò": "o",
        "ù": "u"
    }
    for old, new in replacements.items():
        s = s.replace(old, new)
    s = re.sub(' +', ' ', s)    
    s = s.strip()
    return s

input_24['debitore'] = input_24['debitore'].apply(clean_string)
input_24['Cliente'] = input_24['Cliente'].apply(clean_string)

# SCRAPER

In [12]:
def click_button(driver, button):
    try:
        button.click()
    except Exception as e:
        # If clicking using regular method fails, try clicking using JavaScript
        driver.execute_script("arguments[0].click();", button)

def save_dict_to_json(dictionary, path):
    with open(path, 'w') as file:
        json.dump(dictionary, file)  

In [13]:
vuote = input_24[input_24['CF'].isna()]
input24 = input_24[~input_24['CF'].isna()] 

# SCRAPER F24 CON TUTTO 

In [ ]:
iren = input24[input24['Cliente']=='iren']
input2 = input24[input24['Cliente']!='iren']

Clienti non iren

In [24]:
def select_option(driver, dropdown_locator, option_text):
    try:
        dropdown_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(dropdown_locator))
        dropdown_element.click()
        
        option_locator = (By.XPATH, f'//option[contains(text(), "{option_text}")]')   # ?
        option_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(option_locator))
        option_element.click()
    except Exception as e:
        print(f"Error: {e}")
        raise
       

def compila(driver,office_name,ente,anno,natura,numero,cf,cognome,nome,tipo,debitore):
    url = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm?passo=0'
    driver.get(url)

    try:
        #  inserire l'ufficio
        select_option(driver, (By.CSS_SELECTOR, "select#ufficio.form-control"), office_name)
        #clicca avanti
        select_locator = (By.CSS_SELECTOR, "input#avanti.btn.btn-primary")
        select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
        click_button(driver, select_element)
        time.sleep(1)

        try:
            # clicca per inserire l'ente
            select_option(driver, (By.CSS_SELECTOR, "select#ente.form-control"), ente)
            # inserire l'anno
            select_option(driver, (By.CSS_SELECTOR, "select#anno.form-control"), anno)
            # inserire la natura
            select_option(driver, (By.CSS_SELECTOR, "select#natura.form-control"), natura)


            dropdown_element= WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select#natura.form-control")))
            driver.execute_script("arguments[0].blur();", dropdown_element )
            
            # clicca per inserire il numero
            select_locator = (By.CSS_SELECTOR, "input#numero.form-control")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            # driver.execute_script("arguments[0].scrollIntoView(true);", select_element)
            driver.execute_script("arguments[0].value = arguments[1];", select_element, numero)
                   
            time.sleep(2)

            #clicca avanti
            select_locator = (By.CSS_SELECTOR, "input#avanti.btn.btn-primary")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            click_button(driver, select_element)
            time.sleep(2)

            # clicca su compila f24
            compila_f24_locator = (By.XPATH, "//a[contains(text(), 'Compila F24')]")
            compila_f24_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(compila_f24_locator))
            compila_f24_element.click()

            time.sleep(2)

            # clicca per inserire il codice fiscale
            select_locator = (By.CSS_SELECTOR, "input#codicefiscale")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            # driver.execute_script("arguments[0].scrollIntoView(true);", select_element)
            driver.execute_script("arguments[0].value = arguments[1];", select_element, cf)

            # clicca per inserire il cognome
            select_locator = (By.CSS_SELECTOR, "input#cognome")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            # driver.execute_script("arguments[0].scrollIntoView(true);", select_element)
            driver.execute_script("arguments[0].value = arguments[1];", select_element, cognome)

            # clicca per inserire il nome
            select_locator = (By.CSS_SELECTOR, "input#nome")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            # driver.execute_script("arguments[0].scrollIntoView(true);", select_element)
            driver.execute_script("arguments[0].value = arguments[1];", select_element, nome)

            time.sleep(1)

            download_dir = os.path.join(os.path.expanduser("~"), "Downloads")
            existing_files = set(glob.glob(os.path.join(download_dir, "*")))

            downloaded_file_path = None
            timeout = 30  # Maximum time to wait for download in seconds
            start_time = time.time()

            stampaf24_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "stampaf24button")))
            stampaf24_button.click()

            time.sleep(3)

            while True:
                # Look for new files in the download directory
                new_files = set(glob.glob(os.path.join(download_dir, "*"))) - existing_files
                if new_files:
                    # Get the most recent file that does not have a .crdownload extension
                    completed_files = [f for f in new_files if not f.endswith('.crdownload')]
                    if completed_files:
                        downloaded_file_path = max(completed_files, key=os.path.getctime)
                        break
                if time.time() - start_time > timeout:
                    print("Timeout: File download took too long.")
                    break
                time.sleep(1)

            if downloaded_file_path:
                new_file_name = f"F24_{debitore}_{tipo}.pdf"
                target_dir = f'F24_Download/{today}/Clienti'
                os.makedirs(target_dir, exist_ok=True)
                new_file_path = os.path.join(target_dir, new_file_name)
                os.rename(downloaded_file_path, new_file_path)
                print("File downloaded and moved to:", new_file_path)
            else:
                print("No files were downloaded.")
                
        except Exception as e:
            print(f"Error: {e}")
            return []   

    except Exception as e:
        print(f"Error: {e}")
        return []
   

def main(row):
    options = webdriver.EdgeOptions()
    driver = webdriver.Edge(options=options)
    driver.maximize_window()
    
    compila(
        driver=driver, 
        office_name=row['ufficio'],
        ente=row['emesso da'],
        anno=row['anno'],
        natura=row['natura'],
        numero=row['numero'],
        cf=row['CF'],
        cognome=row['INTESTAZONE F23 PAGAMENTO'],
        nome=row['nome'],
        tipo=row['tipo'],
        debitore=row['debitore']
    ) 
    driver.quit()

 
if __name__ == "__main__":
    input2.apply(main, axis=1)

File downloaded and moved to: F24_Download/2024-06-11/Iren\F24_RICAGNO PAOLO_DECRETO.pdf
File downloaded and moved to: F24_Download/2024-06-11/Iren\F24_SIDIBE SAMBA TENIN_DECRETO.pdf


Cliente iren

In [ ]:
def select_option(driver, dropdown_locator, option_text):
    try:
        dropdown_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(dropdown_locator))
        dropdown_element.click()
        
        option_locator = (By.XPATH, f'//option[contains(text(), "{option_text}")]')   # ?
        option_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(option_locator))
        option_element.click()
    except Exception as e:
        print(f"Error: {e}")
        raise
       

def compila(driver,office_name,ente,anno,natura,numero,cf,cognome,nome,tipo,debitore):
    url = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm?passo=0'
    driver.get(url)

    try:
        select_option(driver, (By.CSS_SELECTOR, "select#ufficio.form-control"), office_name)
        select_locator = (By.CSS_SELECTOR, "input#avanti.btn.btn-primary")
        select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
        click_button(driver, select_element)
        time.sleep(1)

        try:
            select_option(driver, (By.CSS_SELECTOR, "select#ente.form-control"), ente)
            select_option(driver, (By.CSS_SELECTOR, "select#anno.form-control"), anno)
            select_option(driver, (By.CSS_SELECTOR, "select#natura.form-control"), natura)

            dropdown_element= WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select#natura.form-control")))
            driver.execute_script("arguments[0].blur();", dropdown_element )
            
            select_locator = (By.CSS_SELECTOR, "input#numero.form-control")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            driver.execute_script("arguments[0].value = arguments[1];", select_element, numero)
                   
            time.sleep(2)

            select_locator = (By.CSS_SELECTOR, "input#avanti.btn.btn-primary")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            click_button(driver, select_element)
            time.sleep(2)

            compila_f24_locator = (By.XPATH, "//a[contains(text(), 'Compila F24')]")
            compila_f24_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(compila_f24_locator))
            compila_f24_element.click()

            time.sleep(2)

            select_locator = (By.CSS_SELECTOR, "input#codicefiscale")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            driver.execute_script("arguments[0].value = arguments[1];", select_element, cf)

            select_locator = (By.CSS_SELECTOR, "input#cognome")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            driver.execute_script("arguments[0].value = arguments[1];", select_element, cognome)

            # clicca per inserire il nome
            select_locator = (By.CSS_SELECTOR, "input#nome")
            select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(select_locator))
            # driver.execute_script("arguments[0].scrollIntoView(true);", select_element)
            driver.execute_script("arguments[0].value = arguments[1];", select_element, nome)

            time.sleep(1)

            download_dir = os.path.join(os.path.expanduser("~"), "Downloads")
            existing_files = set(glob.glob(os.path.join(download_dir, "*")))

            downloaded_file_path = None
            timeout = 30  # Maximum time to wait for download in seconds
            start_time = time.time()

            stampaf24_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "stampaf24button")))
            stampaf24_button.click()

            time.sleep(3)

            while True:
                new_files = set(glob.glob(os.path.join(download_dir, "*"))) - existing_files
                if new_files:
                    # Get the most recent file that does not have a .crdownload extension
                    completed_files = [f for f in new_files if not f.endswith('.crdownload')]
                    if completed_files:
                        downloaded_file_path = max(completed_files, key=os.path.getctime)
                        break
                if time.time() - start_time > timeout:
                    print("Timeout: File download took too long.")
                    break
                time.sleep(1)

            if downloaded_file_path:
                new_file_name = f"F24_{debitore}_{tipo}.pdf"
                target_dir = f'F24_Download/{today}/Iren'
                os.makedirs(target_dir, exist_ok=True)
                new_file_path = os.path.join(target_dir, new_file_name)
                os.rename(downloaded_file_path, new_file_path)
                print("File downloaded and moved to:", new_file_path)
            else:
                print("No files were downloaded.")
                
        except Exception as e:
            print(f"Error: {e}")
            return []   

    except Exception as e:
        print(f"Error: {e}")
        return []
   

def main(row):
    options = webdriver.EdgeOptions()
    driver = webdriver.Edge(options=options)
    driver.maximize_window()
    
    compila(
        driver=driver, 
        office_name=row['ufficio'],
        ente=row['emesso da'],
        anno=row['anno'],
        natura=row['natura'],
        numero=row['numero'],
        cf=row['CF'],
        cognome=row['INTESTAZONE F23 PAGAMENTO'],
        nome=row['nome'],
        tipo=row['tipo'],
        debitore=row['debitore']
    ) 
    driver.quit()

 
if __name__ == "__main__":
    iren.apply(main,axis=1)
